# New baseline for the bovine with Transformers

inspired from this guide :
https://keras.io/examples/vision/video_transformers/

In [1]:
import matplotlib.pyplot as plt

In [2]:
from problem import get_train_data, get_test_data, WeightedClassificationError

In [3]:
import numpy as np
#CPU
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"  
import tensorflow as tf
from time import time
import pandas as pd
import math

In [ ]:
import io
import imageio
import ipywidgets

In [4]:
from tensorflow.keras import layers
from tensorflow import keras

In [ ]:
# Setting seed for reproducibility
SEED = 42
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
keras.utils.set_random_seed(SEED)

## Define hyperparams

In [5]:
# DATA
BATCH_SIZE = 32
AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = (28, 28, 28, 1)
NUM_CLASSES = 11

# OPTIMIZER
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5

# TRAINING
EPOCHS = 60

# TUBELET EMBEDDING
PATCH_SIZE = (8, 8, 8)
NUM_PATCHES = (INPUT_SHAPE[0] // PATCH_SIZE[0]) ** 2

# ViViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 128
NUM_HEADS = 8
NUM_LAYERS = 8


## Data preparation

In [6]:
videos_train, labels_train = get_train_data()

In [7]:
X_for_classifier= np.array(videos_train)
y_for_classifier= labels_train

In [8]:
videos_test, labels_test  = get_test_data()

In [22]:
Xtest_for_classifier = np.array(videos_test)
ytest_for_classifier = labels_test

In [14]:
from PIL import Image

def resize_frames(video, new_size):
    res=[]
    for frame in video:
        resized_img=Image.fromarray(frame).resize(size=(224, 224))
        res.append(np.array(resized_img))
    return np.array(res)

In [89]:
#function that gets all dataset
# 30 frames per video for 177 video = 2.65 gb !if considering each frame of float64
# as uint8 it takes 0.33 gb

def gen_videos(videolist):
    newvideos=[] # 177*30*250*250
    for video in videolist:
        reducedvideo= video.read_samples(video.frame_times[0:299:10])
        reducedvideo= reducedvideo.astype('uint8')
        #add dimnesion this takes quite a bit of memory ???? dim= 30*250*250*3
        reducedvideo=np.repeat(reducedvideo[...,np.newaxis], 3, -1)
        
        #CROP from 250 to 224()DenseNet121 standards !! TODO !
        reducedvideo=resize_frames(reducedvideo)
        #and add a batch dimension. dim= 1*30*250*250*3
        #reducedvideo = reducedvideo[None, ...]

        newvideos.append(reducedvideo)
    return newvideos

In [90]:
X_for_classifier= np.array(gen_videos(X_for_classifier))
X_for_classifier.shape

(177, 30, 224, 224, 3)

In [91]:
Xtest_for_classifier= np.array(gen_videos(Xtest_for_classifier))
Xtest_for_classifier.shape

(100, 30, 224, 224, 3)

In [92]:
def class_to_int(argument):
    switcher = {
        'A':0,
        'B':1,
        'C':2,
        'D':3,
        'E':4,
        'F':5,
        'G':6,
        'H':7,
    }
 
    # get() method of dictionary data type returns
    # value of passed argument if it is present
    # in dictionary otherwise second argument will
    # be assigned as default value of passed argument
    return switcher.get(argument, "nothing")
 

In [93]:
func=np.vectorize(class_to_int)
#Train
train_labels=func(y_for_classifier)
#Test
test_labels=func(ytest_for_classifier)

In [94]:
train_labels

array([7, 7, 0, 5, 7, 0, 0, 5, 5, 0, 2, 1, 2, 5, 7, 5, 5, 7, 1, 1, 7, 2,
       7, 1, 1, 5, 7, 7, 7, 2, 2, 4, 6, 3, 4, 2, 0, 0, 7, 7, 6, 2, 5, 2,
       6, 1, 7, 7, 7, 2, 7, 7, 7, 6, 0, 6, 0, 5, 4, 0, 5, 6, 5, 4, 5, 4,
       5, 0, 1, 0, 6, 4, 6, 5, 6, 1, 1, 4, 5, 5, 1, 6, 5, 3, 1, 0, 3, 6,
       3, 5, 5, 2, 1, 6, 2, 2, 2, 3, 6, 1, 0, 0, 5, 2, 2, 1, 2, 1, 3, 5,
       6, 4, 1, 2, 1, 3, 3, 1, 0, 7, 7, 5, 7, 7, 7, 5, 7, 6, 4, 5, 0, 7,
       6, 7, 5, 7, 7, 6, 1, 1, 7, 7, 6, 5, 7, 7, 4, 5, 7, 5, 5, 7, 5, 5,
       5, 7, 7, 6, 0, 7, 1, 7, 6, 7, 6, 0, 7, 7, 7, 7, 4, 1, 2, 6, 7, 3,
       3])

In [95]:
test_labels

array([0, 0, 6, 0, 5, 2, 7, 2, 2, 7, 7, 5, 7, 7, 4, 4, 4, 2, 2, 6, 7, 7,
       7, 4, 6, 4, 3, 3, 6, 6, 5, 6, 4, 2, 4, 0, 4, 4, 2, 6, 6, 4, 0, 3,
       3, 3, 0, 0, 5, 2, 1, 5, 2, 1, 1, 1, 2, 4, 4, 1, 4, 3, 6, 4, 1, 6,
       4, 3, 3, 3, 0, 4, 4, 2, 6, 7, 7, 7, 5, 7, 6, 1, 5, 7, 7, 6, 6, 4,
       2, 7, 0, 0, 7, 1, 6, 6, 7, 4, 3, 3])

## Loading test data

In [117]:
 print(tf. __version__) 

2.9.0


In [114]:
videos_test, labels_test  = get_test_data()

In [115]:
#videos_test, labels_test=filter(filters,videos_test, labels_test)



In [116]:
builtx, builty= create_dataset(290,300, videos_test, labels_test)

NameError: name 'create_dataset' is not defined

In [ ]:
Xtest_for_classifier = np.array(builtx)
ytest_for_classifier = np.array(builty)

In [ ]:
ytest_for_classifier

In [ ]:
grayscale_batch=Xtest_for_classifier
rgb_batch = np.repeat(grayscale_batch[..., np.newaxis], 3, -1)

In [ ]:
ytest_for_classifier=func(ytest_for_classifier)

In [ ]:
ytest_for_classifier

In [ ]:
Xtest_for_classifier= rgb_batch

In [ ]:
Xtest_for_classifier.shape

In [ ]:
loss, accuracy = model.evaluate(Xtest_for_classifier, ytest_for_classifier)
print('Test accuracy :', accuracy)

In [ ]:
preds = model.predict(Xtest_for_classifier)
preds = np.argmax(preds, axis=1)
preds

In [ ]:
ytest_for_classifier

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
con_mat = confusion_matrix(ytest_for_classifier, preds)
disp = ConfusionMatrixDisplay(confusion_matrix=con_mat)
disp.plot()
plt.show()

In [ ]:
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1']


In [ ]:
print(classification_report(ytest_for_classifier, preds,labels=[0,1]))